# Data Cleaning Case Study1

Data wrangling / Data munging

Data wrangling , sometimes referred to as data munging, is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics.

The data transformations are typically applied to distinct entities (e.g. fields, rows, columns, data values etc.) within a data set, and could include such actions as extractions, parsing, joining, standardizing, augmenting, cleansing, consolidating and filtering to create desired wrangling outputs that can be leveraged downstream.

The recipients could be individuals, such as data architects or data scientists who will investigate the data further, business users who will consume the data directly in reports, or systems that will further process the data and write it into targets such as data warehouses, data lakes or downstream applications.

Data munging broadly consists of following steps :

* Data Ingestion
* Data merging
* Quick data exploration
* Checking initial stats of the data
* Cleaning the data 
    * check for missing values
    * Treat missing values
    * check for inconsistencies in names / values across data columns
    * Treat data inconsistencies
    * renaming columns names
    * introduce new columns after cleaning
* Exploratory Data Analysis (EDA)
    * Data visualisation
    * Data distribution checking
    * Detailed data stats study
    * Check and treat outliers
* Feature engineering 
    * Binning
    * categorical features to one hot feature conversion
    * categorical features to factorized features
    * creating interactive features
    * creating polynomial features
    * deriving features
    * acquiring more features
* Preprocesssing 
    * feature scaling
    * feature normalization 
    * training data format changing as per ML algo 

## Import libraries

In [1]:
import pandas as pd
import numpy as np

## Data input

In [2]:
# Download
DATASET = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names",
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
)

In [3]:
# Load Training and Test Data Sets
headers = ['age', 'workclass', 'fnlwgt', 
           'education', 'education-num', 
           'marital-status', 'occupation', 
           'relationship', 'race', 'sex', 
           'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 
           'predclass']
training_raw = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                       header=None, 
                       names=headers,na_values=["?"])
test_raw = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', 
                      header=None, 
                      names=headers,na_values=["?"])

In [4]:
# Join Datasets
dataset_raw = training_raw.append(test_raw)
dataset_raw.reset_index(inplace=True)
dataset_raw.drop('index',inplace=True,axis=1)

## Data intial stats checking

In [5]:
dataset_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48843 entries, 0 to 48842
Data columns (total 15 columns):
age               48843 non-null object
workclass         48842 non-null object
fnlwgt            48842 non-null float64
education         48842 non-null object
education-num     48842 non-null float64
marital-status    48842 non-null object
occupation        48842 non-null object
relationship      48842 non-null object
race              48842 non-null object
sex               48842 non-null object
capital-gain      48842 non-null float64
capital-loss      48842 non-null float64
hours-per-week    48842 non-null float64
native-country    48842 non-null object
predclass         48842 non-null object
dtypes: float64(5), object(10)
memory usage: 5.6+ MB


In [6]:
# Describing all the Numerical Features
dataset_raw.describe()

,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [7]:
# Describing all the Categorical Features
dataset_raw.describe(include=['O'])

,age,workclass,education,marital-status,occupation,relationship,race,sex,native-country,predclass
count,48843,48842,48842,48842,48842,48842,48842,48842,48842,48842
unique,147,9,16,7,15,6,5,2,42,4
top,36,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,898,33906,15784,22379,6172,19716,41762,32650,43832,24720


## Q1) Check for missing values

In [8]:
missing_value_by_column=dataset_raw.isnull().sum()
missing_value_by_column

age               0
workclass         1
fnlwgt            1
education         1
education-num     1
marital-status    1
occupation        1
relationship      1
race              1
sex               1
capital-gain      1
capital-loss      1
hours-per-week    1
native-country    1
predclass         1
dtype: int64

## Q2) For columns with missing values, print the number of rows with missing values

In [9]:
#number of rows with missing values
dataset_raw.isnull().any(axis=1).sum()


1

In [10]:
#Row with missing value
missing_row = dataset_raw[dataset_raw.isna().any(axis=1)]
missing_row

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,predclass
32561,|1x3 Cross validator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Q3) For columns with missing values, print the percentage of rows with missing values

In [11]:
dataset_raw.isna().mean().round(4) * 100

age               0.0
workclass         0.0
fnlwgt            0.0
education         0.0
education-num     0.0
marital-status    0.0
occupation        0.0
relationship      0.0
race              0.0
sex               0.0
capital-gain      0.0
capital-loss      0.0
hours-per-week    0.0
native-country    0.0
predclass         0.0
dtype: float64

***

In [12]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

## Q4) Use the DataFrameImputer class defined above to impute values to rows with  missing values.

In [13]:
xt = DataFrameImputer().fit_transform(dataset_raw)
xt[xt.isnull().any(axis=1)].count()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
predclass         0
dtype: int64

In [14]:
xt.iloc[32561]

age               |1x3 Cross validator
workclass                      Private
fnlwgt                          189664
education                      HS-grad
education-num                  10.0781
marital-status      Married-civ-spouse
occupation              Prof-specialty
relationship                   Husband
race                             White
sex                               Male
capital-gain                   1079.07
capital-loss                   87.5023
hours-per-week                 40.4224
native-country           United-States
predclass                        <=50K
Name: 32561, dtype: object

## Q5) Use appropriate formatting for all columns

In [15]:
dataset_raw['occupation'].unique()
dataset_raw['occupation'] = dataset_raw['occupation'].apply(lambda x : str(x).strip())

In [16]:
dataset_raw.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,predclass
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [17]:
#changing column names to capital
dataset_raw.columns = map(lambda x: str(x).upper(), dataset_raw.columns)
dataset_raw.head(3)

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION-NUM,MARITAL-STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL-GAIN,CAPITAL-LOSS,HOURS-PER-WEEK,NATIVE-COUNTRY,PREDCLASS
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K


In [18]:
dataset_raw.keys()

Index(['AGE', 'WORKCLASS', 'FNLWGT', 'EDUCATION', 'EDUCATION-NUM',
       'MARITAL-STATUS', 'OCCUPATION', 'RELATIONSHIP', 'RACE', 'SEX',
       'CAPITAL-GAIN', 'CAPITAL-LOSS', 'HOURS-PER-WEEK', 'NATIVE-COUNTRY',
       'PREDCLASS'],
      dtype='object')

***